In [3]:
import pandas as pd
import statsmodels.api as sm

# Load the uploaded datasets
tesla_close = pd.read_csv("tesla_close_2014_2024.csv")
sp500_close = pd.read_csv("sp500_close_2014_2024.csv")
income_statement = pd.read_csv("tesla_income_statement_2022_2024.csv")
balance_sheet = pd.read_csv("tesla_balance_sheet_2022_2024.csv")
cashflow_statement = pd.read_csv("tesla_cashflow_statement_2022_2024.csv")

# Display first few rows of each to understand structure
# tesla_close.head(), sp500_close.head(), income_statement.head(), balance_sheet.head(), cashflow_statement.head()


In [5]:
# Clean and align the Tesla and S&P 500 datasets by date for Beta calculation

# Step 1: Clean column names and parse dates
tesla_close.columns = ["Date", "TSLA"]
sp500_close = sp500_close.iloc[2:]  # Skip metadata rows
sp500_close.columns = ["Date", "SP500"]
sp500_close["Date"] = pd.to_datetime(sp500_close["Date"])
tesla_close["Date"] = pd.to_datetime(tesla_close["Date"])

# Step 2: Merge the two datasets on date
merged = pd.merge(tesla_close, sp500_close, on="Date", how="inner")
merged = merged.sort_values("Date")

# Fix data type issues by ensuring TSLA and SP500 columns are numeric
merged["TSLA"] = pd.to_numeric(merged["TSLA"], errors="coerce")
merged["SP500"] = pd.to_numeric(merged["SP500"], errors="coerce")

# Drop rows with NaN values due to coercion or missing data
merged = merged.dropna()

# Recalculate returns
merged["TSLA_return"] = merged["TSLA"].pct_change()
merged["SP500_return"] = merged["SP500"].pct_change()
merged = merged.dropna()

# Regression for Beta
X = sm.add_constant(merged["SP500_return"])
y = merged["TSLA_return"]
model = sm.OLS(y, X).fit()
beta_value = model.params["SP500_return"]
intercept = model.params["const"]

beta_value, intercept, model.rsquared



(0.6377966635219516, 0.01506643524553374, 0.0017697335038198148)

In [10]:
# Average annual market return from S&P
merged['Year'] = merged['Date'].dt.year
sp500_yearly = merged.groupby('Year').agg(
    start_price=('SP500', 'first'),
    end_price=('SP500', 'last')
)
sp500_yearly['annual_return'] = (sp500_yearly['end_price'] - sp500_yearly['start_price']) / sp500_yearly['start_price']
average_market_return = sp500_yearly['annual_return'].mean()

# CAPM Cost of Equity
risk_free_rate = 0.042
cost_of_equity = risk_free_rate + beta_value * (average_market_return - risk_free_rate)
{
    "Beta": beta_value,
    "Average Market Return (S&P 500)": average_market_return,
    "CAPM Cost of Equity": cost_of_equity
 }

{'Beta': 0.6377966635219516,
 'Average Market Return (S&P 500)': 0.12580129798146134,
 'CAPM Cost of Equity': 0.0954481882513849}

In [14]:
# WACC
latest_balance = balance_sheet.iloc[0]
latest_income = income_statement.iloc[0]
latest_cashflow = cashflow_statement.iloc[0]

total_debt = latest_balance["Total Debt"]
market_price_assumption = 239.43  # current market price
equity_market_value = latest_balance["Ordinary Shares Number"] * market_price_assumption
debt_market_value = total_debt
E = equity_market_value
D = debt_market_value
V = E + D

interest_expense = latest_income["Interest Expense Non Operating"]
cost_of_debt = interest_expense / total_debt
tax_rate = latest_income["Tax Rate For Calcs"]

wacc = (E / V) * cost_of_equity + (D / V) * cost_of_debt * (1 - tax_rate)

# Intrinsic Value per Share using Gordon Growth Model
fcf = latest_cashflow["Free Cash Flow"]
shares_outstanding = latest_balance["Ordinary Shares Number"]
g = 0.04  # growth rate

value_of_firm = fcf * (1 + g) / (wacc - g)
intrinsic_value_per_share = value_of_firm / shares_outstanding

# Output results
{
    "FCF": fcf,
    "WACC": wacc,
    "Intrinsic Value per Share": intrinsic_value_per_share,
    "R-squared (Beta Fit)": model.rsquared
}

{'FCF': 3581000000.0,
 'WACC': 0.09414618242620042,
 'Intrinsic Value per Share': 21.387192484881325,
 'R-squared (Beta Fit)': 0.0017697335038198148}

Market Price : $239.43

Intrinsic Value (DCF-derived): $21.39
239.43/21.39 =11.193

The market is pricing Tesla at nearly 11x the value of the stock  based on current Free Cash Flow, WACC, and long-term growth assumptions.